In [7]:
import pandas as pd
print('hello')

hello


In [8]:
# importing the tensorflow module
import tensorflow as tf

# importing the training and testing dataset
(x_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

2022-07-06 15:03:05.496519: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-06 15:03:05.496560: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
# Normalize pixels to values between 0 and 1
x_train = x_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [10]:
# Building  model
model = tf.keras.Sequential()

# flattening the layer
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

# defining hidden layer with 200 nodes
model.add(tf.keras.layers.Dense(units=200, activation='relu'))

# adding a drop out in the hidden layer
model.add(tf.keras.layers.Dropout(0.2))

# adding output laers
model.add(tf.keras.layers.Dense(10, activation='softmax'))

2022-07-06 15:04:35.466183: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-06 15:04:35.466295: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-06 15:04:35.466364: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (UCA-Nar): /proc/driver/nvidia/version does not exist
2022-07-06 15:04:35.484149: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# compiling the model with learning rate is 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              
#               loss function is categorical cross entropy
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

/home/uca/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [12]:
#  Early stopping set after 4 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

# Training the model using 10 epochs
model.fit(x_train, y_train, epochs=10, validation_split=0.2, callbacks=[stop_early], verbose=2)

2022-07-06 15:04:41.084835: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 150528000 exceeds 10% of free system memory.


Epoch 1/10


2022-07-06 15:04:51.132305: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 37632000 exceeds 10% of free system memory.


1500/1500 - 8s - loss: 0.5386 - accuracy: 0.8104 - val_loss: 0.4178 - val_accuracy: 0.8507 - 8s/epoch - 5ms/step
Epoch 2/10
1500/1500 - 4s - loss: 0.4033 - accuracy: 0.8525 - val_loss: 0.3806 - val_accuracy: 0.8597 - 4s/epoch - 3ms/step
Epoch 3/10
1500/1500 - 4s - loss: 0.3638 - accuracy: 0.8671 - val_loss: 0.3727 - val_accuracy: 0.8642 - 4s/epoch - 3ms/step
Epoch 4/10
1500/1500 - 4s - loss: 0.3428 - accuracy: 0.8726 - val_loss: 0.3477 - val_accuracy: 0.8750 - 4s/epoch - 3ms/step
Epoch 5/10
1500/1500 - 4s - loss: 0.3259 - accuracy: 0.8801 - val_loss: 0.3331 - val_accuracy: 0.8799 - 4s/epoch - 3ms/step
Epoch 6/10
1500/1500 - 5s - loss: 0.3119 - accuracy: 0.8850 - val_loss: 0.3223 - val_accuracy: 0.8835 - 5s/epoch - 3ms/step
Epoch 7/10
1500/1500 - 5s - loss: 0.3006 - accuracy: 0.8890 - val_loss: 0.3309 - val_accuracy: 0.8800 - 5s/epoch - 3ms/step
Epoch 8/10
1500/1500 - 4s - loss: 0.2918 - accuracy: 0.8910 - val_loss: 0.3293 - val_accuracy: 0.8836 - 4s/epoch - 3ms/step
Epoch 9/10
1500/150

In [13]:
# importing the pandas module
import pandas as pd

# defining the function
def evaluate_model(model, X_test, y_test):

    # Evauting the model using teh testing dataset
    eval_dict = model.evaluate(X_test, y_test, return_dict=True)
    
    # converting the evaluation ( accuracy and loss) to pandas dataframe
    result = pd.DataFrame([eval_dict.values()], columns=[list(eval_dict.keys())])
    
    # returining dataframe
    return result

# calling the funtion
results = evaluate_model(model, X_test, y_test)


# Display results
results.head()

2022-07-06 15:08:05.389686: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 31360000 exceeds 10% of free system memory.


313/313 [==============================] - 2s 3ms/step - loss: 0.3455 - accuracy: 0.8769


,loss,accuracy
0,0.345537,0.8769


In [14]:
# creating function
def build_model(hyper_parameter):

    # building the model
    model = tf.keras.Sequential()

    # flattenting the images
    model.add(tf.keras.layers.Flatten(input_shape=(28,28)))

    # tunning hidden layers (1-3), units ( 50 -300),
    for i in range(1, hyper_parameter.Int("num_layers", 2, 4)):
        model.add(
            tf.keras.layers.Dense(
                units=hyper_parameter.Int("units_" + str(i), min_value=50, max_value=300, step=30),
                activation="relu")
            )
        
        # drop out layer with values form 0-0.3
        model.add(tf.keras.layers.Dropout(hyper_parameter.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    
    # Add output layer.
    model.add(tf.keras.layers.Dense(units=10, activation="softmax"))
    
    # Tune learning from 0.01, 0.001
    hyper_parameter_learning_rate = hyper_parameter.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    # Define optimizer, loss, and metrics
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hyper_parameter_learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=["accuracy"])
    
    return model

In [15]:
# importing the karas tuner
import kerastuner as kt

# Instantiate the tuner
tuner = kt.Hyperband(build_model,
                     objective="val_accuracy",
                     max_epochs=20,
                     factor=3,
                     hyperband_iterations=10,
                     directory="kt_dir",
                     project_name="kt_hyperband",)

/tmp/ipykernel_6008/3583274274.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [17]:

# summary of tuner
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 4, 'step': 1, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 300, 'step': 30, 'sampling': None}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [18]:
# early stop function
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [19]:
# Tunner search
tuner.search(x_train, y_train, epochs=20, validation_split=0.2, callbacks=[stop_early], verbose=2)


Trial 300 Complete [00h 01m 09s]
val_accuracy: 0.887583315372467

Best val_accuracy So Far: 0.8944166898727417
Total elapsed time: 02h 27m 59s
INFO:tensorflow:Oracle triggered exit


In [20]:

# Get the optimum values for parameters
best_hps=tuner.get_best_hyperparameters()[0]

# Build model on optimum paramters
optimum_model = tuner.hypermodel.build(best_hps)

# Train the hypertuned model
optimum_model.fit(x_train, y_train, epochs=10, validation_split=0.2, callbacks=[stop_early], verbose=2)

Epoch 1/10
1500/1500 - 7s - loss: 0.7676 - accuracy: 0.7398 - val_loss: 0.4772 - val_accuracy: 0.8371 - 7s/epoch - 4ms/step
Epoch 2/10
1500/1500 - 5s - loss: 0.4711 - accuracy: 0.8360 - val_loss: 0.4218 - val_accuracy: 0.8508 - 5s/epoch - 4ms/step
Epoch 3/10
1500/1500 - 6s - loss: 0.4166 - accuracy: 0.8539 - val_loss: 0.3831 - val_accuracy: 0.8654 - 6s/epoch - 4ms/step
Epoch 4/10
1500/1500 - 5s - loss: 0.3814 - accuracy: 0.8676 - val_loss: 0.3782 - val_accuracy: 0.8643 - 5s/epoch - 3ms/step
Epoch 5/10
1500/1500 - 5s - loss: 0.3575 - accuracy: 0.8730 - val_loss: 0.3504 - val_accuracy: 0.8756 - 5s/epoch - 3ms/step
Epoch 6/10
1500/1500 - 5s - loss: 0.3410 - accuracy: 0.8792 - val_loss: 0.3510 - val_accuracy: 0.8753 - 5s/epoch - 4ms/step
Epoch 7/10
1500/1500 - 5s - loss: 0.3274 - accuracy: 0.8813 - val_loss: 0.3372 - val_accuracy: 0.8794 - 5s/epoch - 3ms/step
Epoch 8/10
1500/1500 - 6s - loss: 0.3122 - accuracy: 0.8866 - val_loss: 0.3446 - val_accuracy: 0.8776 - 6s/epoch - 4ms/step
Epoch 9/

In [24]:
# get te optimum models results
hyper_df = evaluate_model(optimum_model, X_test, y_test)

# printing the accuracy
hyper_df

313/313 [==============================] - 1s 2ms/step - loss: 0.3435 - accuracy: 0.8787


,loss,accuracy
0,0.343453,0.8787


In [25]:
best_hps

In [27]:

# creating model with optimimum parameters
optimum_model = tuner.hypermodel.build(best_hps)

# history of the model
history = optimum_model.fit(x_train, y_train, epochs=200, validation_split=0.2)

# fining the optimum epochs
val_acc_per_epoch = history.history['val_accuracy']

# printing the best epoch value
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch value is: ' ,best_epoch)

Epoch 1/200
1500/1500 [==============================] - 21s 10ms/step - loss: 0.7620 - accuracy: 0.7432 - val_loss: 0.4832 - val_accuracy: 0.8366
Epoch 2/200
1500/1500 [==============================] - 10s 6ms/step - loss: 0.4794 - accuracy: 0.8345 - val_loss: 0.4381 - val_accuracy: 0.8471
Epoch 3/200
1500/1500 [==============================] - 11s 7ms/step - loss: 0.4208 - accuracy: 0.8519 - val_loss: 0.3919 - val_accuracy: 0.8618
Epoch 4/200
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3888 - accuracy: 0.8626 - val_loss: 0.3666 - val_accuracy: 0.8677
Epoch 5/200
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3646 - accuracy: 0.8689 - val_loss: 0.3570 - val_accuracy: 0.8711
Epoch 6/200
1500/1500 [==============================] - 17s 11ms/step - loss: 0.3476 - accuracy: 0.8737 - val_loss: 0.3432 - val_accuracy: 0.8762
Epoch 7/200
1500/1500 [==============================] - 13s 9ms/step - loss: 0.3328 - accuracy: 0.8795 - val_loss: 0.3420

In [28]:
# Train the hypertuned model
optimum_model.fit(x_train, y_train, epochs=70, validation_split=0.2, callbacks=[stop_early], verbose=2)

Epoch 1/70
1500/1500 - 10s - loss: 0.0155 - accuracy: 0.9948 - val_loss: 0.9317 - val_accuracy: 0.8854 - 10s/epoch - 6ms/step
Epoch 2/70
1500/1500 - 7s - loss: 0.0158 - accuracy: 0.9943 - val_loss: 0.9943 - val_accuracy: 0.8926 - 7s/epoch - 5ms/step
Epoch 3/70
1500/1500 - 7s - loss: 0.0150 - accuracy: 0.9948 - val_loss: 0.9364 - val_accuracy: 0.8932 - 7s/epoch - 5ms/step
Epoch 4/70
1500/1500 - 6s - loss: 0.0171 - accuracy: 0.9940 - val_loss: 0.9400 - val_accuracy: 0.8910 - 6s/epoch - 4ms/step
Epoch 5/70
1500/1500 - 6s - loss: 0.0130 - accuracy: 0.9960 - val_loss: 0.9536 - val_accuracy: 0.8916 - 6s/epoch - 4ms/step
Epoch 6/70
1500/1500 - 6s - loss: 0.0159 - accuracy: 0.9945 - val_loss: 0.9933 - val_accuracy: 0.8912 - 6s/epoch - 4ms/step


In [29]:
# get te optimum models results
hyper_df = evaluate_model(optimum_model, X_test, y_test)

# printing the accuracy
hyper_df

313/313 [==============================] - 10s 5ms/step - loss: 1.0371 - accuracy: 0.8931


,loss,accuracy
0,1.037149,0.8931
